In [ ]:
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

#load a pretrained perplextiy model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

In [1]:
#Perplexity Calculation

In [ ]:
def calculate_perplexity(text, stride=512):
    encodings = tokenizer(text, return_tensors="pt")
    nlls = []
    input_ids = encodings.input_ids
    seq_len = input_ids.size(1)

    for i in range(0, seq_len, stride):
        begin_loc = max(i + stride - model.config.n_positions, 0)
        end_loc = min(i + stride, seq_len)
        trg_len = end_loc - i
        input_ids_chunk = input_ids[:, begin_loc:end_loc]
        target_ids = input_ids_chunk.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids_chunk, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

In [ ]:
df_sample_2["Title_ppl"] = df_sample_2["Title"].apply(calculate_perplexity)
df_sample_2["Conclusion_ppl"] = df_sample_2["Conclusion"].apply(calculate_perplexity)

In [ ]:
print(df_sample_2[["Title_ppl", "Conclusion_ppl"]].head())